In [72]:
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import sys
import os
import pandas as pd 
import argparse
import random
from scipy.spatial.transform import Rotation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

# Imposta il seed per numpy
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

custom = True

_size = "500"
dataset = "h36m"
_type = "3D"
ceil = 10 
dim = 2
method = "gt"
name = "completion_murino_"+_type+"_"+dataset+"_" + method +"_mask"+str(ceil)+"_norot" # 0.0411 with +30*



def random_rotation_2d():    
    # Genera un angolo casuale per la rotazione in radianti
    #angle = np.random.uniform(0, 2 * np.pi)
    #angle = np.random.uniform(-np.pi/6,np.pi/6)
    angle=0
    # Crea la matrice di rotazione 2D
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)],
                                [np.sin(angle), np.cos(angle)]])

    return rotation_matrix

def random_rotation_3d():    
    # Genera una rotazione casuale tramite l'uso della libreria `scipy`
    rotation = Rotation.from_euler('xyz', np.random.rand(3), degrees=False)
    
    # Estrai la matrice di rotazione dalla rotazione
    rotation_matrix = rotation.as_matrix()
    
    return rotation_matrix




model_name = '../models/' + name + '.h5'
input_path = "../local_dataset/training/completion/"+_type+"/"

if method is not None:
    X_train = np.load(os.path.join(input_path,"X_train_" + dataset + "_" +  _size +  "_minmax.npy")).swapaxes(1,2)
    X_val = np.load(os.path.join(input_path,"X_val_" + dataset + "_" +  _size +  "_minmax.npy")).swapaxes(1,2)
else:
    X_train = np.load(os.path.join(input_path,"X_train_" + dataset + "_" + method + "_" + _size +  "_minmax.npy")).swapaxes(1,2)
    X_val = np.load(os.path.join(input_path,"X_val_" + dataset + "_" + method + "_" + _size +  "_minmax.npy")).swapaxes(1,2)


if custom:
    X_train = np.load("/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/training_FLK2/h36m_cameraview_3D/X_train_1000.npy")
    X_val = np.load("/home/emartini/nas/MAEVE/HUMAN_MODEL/dataset/training_FLK2/h36m_cameraview_3D/X_val_1000.npy")
    model_name = "tmp/MLP_bio_h36m_cameraview_3D_absolute_onehot.h5"
    dim = 3

#X_val = np.load(os.path.join(input_path,"X_val_aist_" +  _size +  "_minmax.npy")).swapaxes(1,2)
#X_test = np.load(os.path.join(input_path,"X_test_" + dataset + "_" +  _size +  ".npy")).swapaxes(1,2)

#X_train = X_train.swapaxes(1,2)
#X_val = X_val.swapaxes(1,2)

print(X_train[0,:,:])
print(X_val[0,:,:])

[[-364.1775   -975.4356   6021.122    -109.1984   -954.0909   6099.669
  -508.9153   -730.9489   5921.708       6.785923 -680.884    6149.784
  -557.0898   -488.4602   5995.181     -46.69554  -470.2704   6288.714
  -372.7751   -575.7365   5872.574    -103.1269   -597.285    5963.012
  -446.7088   -132.9091   6060.143    -170.6657   -143.0198   6123.739
  -456.9418    323.8216   6125.481    -171.2247    317.4601   6093.172   ]]
[[  90.01911  -712.6233   3312.449      65.28832  -637.2011   3006.589
    12.47191  -532.5627   3516.78        3.935964 -372.9722   2925.318
   171.4691   -390.2531   3389.79      176.4971   -232.0712   3034.999
    34.38879  -260.741    3373.961     -61.70858  -226.4938   3117.111
    85.31891   195.8902   3421.741     -87.43157   221.6817   3226.042
    -5.04759   645.9555   3454.621    -365.6985    473.6731   3492.263   ]]


In [73]:
keypoints = int(X_train.shape[2]/dim) 

# X_train = X_train[:100,:,:]
# X_val = X_val[:100,:,:]

X_train_enhanced = np.repeat(X_train,1,axis=0)


In [74]:
def normalize_bio_root_absolute(dataset):
    from Skeleton import Skeleton,ConstrainedSkeleton
    s12 = Skeleton('BODY12.xml')
    s15 = ConstrainedSkeleton('BODY15_constrained.xml')
    # Normalization using Biomecanichal model
    labels = ['LShoulder','RShoulder','LElbow','RElbow','LWrist','RWrist','LHip','RHip','LKnee','RKnee','LAnkle','RAnkle']
    
    for j in range(dataset.shape[0]):
        x = dataset[j,:,:].reshape(-1,3)
        s12.load_from_numpy(x,labels)
        s15.load_from_BODY12(s12)
        h = s15.estimate_height(False)
        root = s15.to_numpy(["Root"])
        x = (x-root)/h
        dataset[j,:,:] = x.ravel()

normalize_bio_root_absolute(X_train_enhanced)
normalize_bio_root_absolute(X_val)

In [75]:
print("Masking")
y_train = X_train_enhanced.copy()
X_train_enhanced_oneauto = np.zeros( (X_train_enhanced.shape[0], X_train_enhanced.shape[1], keypoints*(dim+1)) )
for r in range(X_train_enhanced.shape[0]):      
    kps_to_mask = np.random.choice(list(range(keypoints)),np.random.randint(low=1,high=ceil+1),replace=False)
    for j in kps_to_mask:
        X_train_enhanced[r,0,dim*j:dim*j+dim] = np.nan
    skeleton = X_train_enhanced[r,0,:].reshape(-1,3)
    skeleton = np.insert(skeleton, 3, values=np.where(np.isnan(skeleton[:,0]), 1, 0), axis=1)
    X_train_enhanced_oneauto[r,0,:] = skeleton.ravel()
    
X_train = X_train_enhanced_oneauto.copy()
# print(X_train)

y_val = X_val.copy()
X_val_enhanced_oneauto = np.zeros( (X_val.shape[0], X_val.shape[1], keypoints*(dim+1)) )
for r in range(X_val.shape[0]):      
        kps_to_mask = np.random.choice(list(range(keypoints)),np.random.randint(low=1,high=ceil+1),replace=False)
        for j in kps_to_mask:
            X_val[r,0,dim*j:dim*j+dim] = np.nan
        skeleton = X_val[r,0,:].reshape(-1,3)
        skeleton = np.insert(skeleton, 3, values=np.where(np.isnan(skeleton[:,0]), 1, 0), axis=1)
        X_val_enhanced_oneauto[r,0,:] = skeleton.ravel()
X_val = X_val_enhanced_oneauto.copy()

Masking
[[[        nan         nan         nan ...         nan         nan
    1.        ]]

 [[        nan         nan         nan ...         nan         nan
    1.        ]]

 [[        nan         nan         nan ...         nan         nan
    1.        ]]

 ...

 [[-0.08766368 -0.13355102 -0.0016917  ...         nan         nan
    1.        ]]

 [[-0.05917127 -0.11206585 -0.09970918 ...  0.63427097  0.28789654
    0.        ]]

 [[ 0.0792622  -0.12775844 -0.02317264 ...  0.6958241   0.09664778
    0.        ]]]


In [76]:
# Creazione del modello Masked Autoencoder
@keras.saving.register_keras_serializable('MLP')
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=keypoints * (dim+1))
        self.layer1 = tf.keras.layers.Dense(128, activation='relu')
        self.layer2 = tf.keras.layers.Dense(256, activation='relu')
        self.layer3 = tf.keras.layers.Dense(128, activation='relu')
        self.layer4 = tf.keras.layers.Dense(keypoints * dim, activation='linear')

    def build(self, input_shape):
        # Definisci i pesi dei layer in base alle dimensioni dell'input
        super(MLP, self).build(input_shape)

    def call(self, inputs):
  
        x = tf.where(tf.math.is_nan(inputs), -1 * tf.ones_like(inputs), inputs)
        x = self.input_layer(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        decoded = self.layer4(x)
        return decoded

def custom_mse_loss(y_true, y_pred):
    mse_class = tf.reduce_mean(tf.square(y_true - y_pred))
    return mse_class


autoencoder = MLP()
autoencoder.build(input_shape=(1, keypoints * (dim+1)))
autoencoder.summary()



autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=custom_mse_loss, metrics=["mae"])
history = autoencoder.fit(X_train.squeeze(), y_train.squeeze(), batch_size=128, epochs=100, shuffle = True, validation_data=(X_val,y_val))
autoencoder.save_weights(model_name)

Model: "mlp_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 48)]              0         
                                                                 
 dense_16 (Dense)            multiple                  6272      
                                                                 
 dense_17 (Dense)            multiple                  33024     
                                                                 
 dense_18 (Dense)            multiple                  32896     
                                                                 
 dense_19 (Dense)            multiple                  4644      
                                                                 
Total params: 76836 (300.14 KB)
Trainable params: 76836 (300.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0034 - mae: 0.0375 - val_loss: 0.0024 - val_mae: 0.0309
Epoch 2/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0023 - mae: 0.0290 - val_loss: 0.0022 - val_mae: 0.0283
Epoch 3/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0021 - mae: 0.0275 - val_loss: 0.0021 - val_mae: 0.0273
Epoch 4/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0020 - mae: 0.0267 - val_loss: 0.0021 - val_mae: 0.0268
Epoch 5/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0020 - mae: 0.0261 - val_loss: 0.0020 - val_mae: 0.0274
Epoch 6/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0019 - mae: 0.0257 - val_loss: 0.0019 - val_mae: 0.0255
Epoch 7/100
5250/5250 [==============================] - 9s 2ms/step - loss: 0.0019 - mae: 0.0254 - val_loss: 0.0019 - val_mae: 0.0258
Epoch 8/100
5250/5250 [==============================] 

In [77]:
autoencoder.summary()
autoencoder.save_weights(model_name)
autoencoder.save(model_name.replace(".h5",""))

Model: "mlp_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 48)]              0         
                                                                 
 dense_16 (Dense)            multiple                  6272      
                                                                 
 dense_17 (Dense)            multiple                  33024     
                                                                 
 dense_18 (Dense)            multiple                  32896     
                                                                 
 dense_19 (Dense)            multiple                  4644      
                                                                 
Total params: 76836 (300.14 KB)
Trainable params: 76836 (300.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
INFO:tensorflow:Assets 

INFO:tensorflow:Assets written to: tmp/MLP_bio_h36m_cameraview_3D_absolute_onehot/assets
